# Development: Impose NACE codes 

Mergeing of leakage lists from different years to get nace codes of installations.

Sources:  

    - 2020: Is from preliminary assessment https://ec.europa.eu/clima/events/stakeholder-meeting-results-preliminary-carbon-leakage-list-phase-4-eu-emissions-trading_en
    - 2015: https://ec.europa.eu/clima/sites/clima/files/installation_nace_rev2_matching_en.xls
    

jab 1.5.2020

# Packages and options

In [1]:
import pandas as pd

In [2]:
fn_leakage_2015 = "./leakage_2015.xls"
fn_leakage_2020 = "./leakage_2020.xlsx"

# Load data and merge lists

In [3]:
df_l15 = pd.read_excel(fn_leakage_2015, na_values="-")
df_l15["id"] = df_l15.COUNTRY_CODE + "_" + df_l15.INSTALLATION_IDENTIFIER.astype("str")
df_l15 = df_l15[["id", "NACE Rev2"]].copy()
df_l15.columns = ["id", "nace15"]
df_l15 = df_l15[df_l15.nace15.notnull()].copy()
#df_l15.nace15 = df_l15.nace15.astype("float")
df_l15.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13365 entries, 0 to 15891
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      13365 non-null  object 
 1   nace15  13365 non-null  float64
dtypes: float64(1), object(1)
memory usage: 313.2+ KB


In [4]:
df_l20 = pd.read_excel(fn_leakage_2020, skiprows=2)
df_l20["id"] = df_l20.COUNTRY_CODE + "_" + df_l20.INSTALLATION_IDENTIFIER.astype("str")
df_l20 = df_l20[["id", "NACE Rev2"]].copy()
df_l20.columns = ["id", "nace20"]
df_l20 = df_l20[df_l20.nace20.notnull()].copy()
df_l20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15068 entries, 0 to 15451
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      15068 non-null  object 
 1   nace20  15068 non-null  float64
dtypes: float64(1), object(1)
memory usage: 353.2+ KB


In [5]:
df_nace = df_l15.merge(df_l20, on="id", how="outer")
df_nace.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15084 entries, 0 to 15083
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      15084 non-null  object 
 1   nace15  13365 non-null  float64
 2   nace20  15076 non-null  float64
dtypes: float64(2), object(1)
memory usage: 471.4+ KB


In [6]:
df_nace["nace"] = df_nace.nace20.fillna(df_nace.nace15)
df_nace.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15084 entries, 0 to 15083
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      15084 non-null  object 
 1   nace15  13365 non-null  float64
 2   nace20  15076 non-null  float64
 3   nace    15084 non-null  float64
dtypes: float64(3), object(1)
memory usage: 589.2+ KB


Modify codes to have always have two digits before the dot

In [7]:
to_adjust = df_nace.nace.map(lambda x: len(str(x).split(".")[0]) < 2)
df_nace.loc[to_adjust, "nace"] = df_nace[to_adjust].nace.map(lambda x: "0" + str(x))

to_adjust = df_nace.nace15.map(lambda x: len(str(x).split(".")[0]) < 2)
df_nace.loc[to_adjust, "nace15"] = df_nace[to_adjust].nace15.map(lambda x: "0" + str(x))

to_adjust = df_nace.nace20.map(lambda x: len(str(x).split(".")[0]) < 2)
df_nace.loc[to_adjust, "nace20"] = df_nace[to_adjust].nace20.map(lambda x: "0" + str(x))


# Analysis

In [8]:
df_nace["is_same"] = df_nace.nace15 == df_nace.nace20
df_nace[(~df_nace["is_same"]) & (df_nace.nace15.notnull())]

,id,nace15,nace20,nace,is_same
14,AT_15,24.1,35.1,35.1,False
82,AT_83,17.12,35.1,35.1,False
579,BE_749,20.14,35.1,35.1,False
614,BG_3,22.11,35.1,35.1,False
714,BG_103,35.11,35.3,35.3,False
...,...,...,...,...,...
12790,SE_532,25.62,35.3,35.3,False
12901,SE_656,25.62,35.3,35.3,False
13007,SE_780,24.1,35.1,35.1,False
13211,SK_53,20.14,35.1,35.1,False


In [9]:
df_nace[(df_nace.nace20.isnull()) & (df_nace.nace15.notnull())]

,id,nace15,nace20,nace,is_same
772,CY_1,35.11,NaN,35.11,False
773,CY_2,35.11,NaN,35.11,False
774,CY_3,35.11,NaN,35.11,False
775,CY_11,23.51,NaN,23.51,False
776,CY_12,23.41,NaN,23.41,False
777,CY_13,23.32,NaN,23.32,False
10098,MT_220000000000001,35.11,NaN,35.11,False
10099,MT_220000000000002,35.11,NaN,35.11,False


So we can see that there is a slight change for some installations (275) which seem to have been moved to 35.3. The new list misses codes for some early installations. We add these from the older list.

# Export

In [10]:
df_nace.to_csv("../nace_leakage.csv", index=False)

In [11]:
df_nace.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15084 entries, 0 to 15083
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       15084 non-null  object
 1   nace15   13365 non-null  object
 2   nace20   15076 non-null  object
 3   nace     15084 non-null  object
 4   is_same  15084 non-null  bool  
dtypes: bool(1), object(4)
memory usage: 603.9+ KB
